# Observational study 

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from networkx import nx
from plotnine import ggplot, aes, geom_bar
from functions import *
import spacy
import time
import datetime
import statsmodels.formula.api as smf
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category= SettingWithCopyWarning)

In [2]:
from popularity import trending_history
from financials import *

In [3]:
elon_org_df = pd.read_csv('Data/org-lg-Elon Musk.csv.bz2')

## Matching : 

In [ ]:
def compareCompanies(companies, tickers, date, shares, hist):
    # Returns a df with the following columns: ['Date', 'Close', 'Volume', 'MarketCap', 'Popularity']
    df = pd.DataFrame(companies, columns=['Name'])
    df['Date'] = date
    
    marketData = []

    for i in range(len(tickers)):
        marketData.append(obs_info(tickers[i], date, shares[i], hist[i]).values.tolist())
    
    df['Close'] = [i[0] for i in marketData]
    df['Volume'] = [i[1] for i in marketData]
    df['MarketCap'] = [i[2] for i in marketData]
    
    df_trend = trending_history(companies, date)
    df_trend_list = df_trend.values.tolist()[0]
    df['Popularity'] = df_trend_list
    
    return df

def obs_info(ticker, date, sharesOutstanding, comp_hist):
    for i in range(len(comp_hist)):
        if comp_hist.iloc[i].name.date() > datetime.date.fromisoformat(date):
            df = comp_hist.iloc[i]
            break
    df['MarketCap'] = int (sharesOutstanding or 0)*df['Close']
    
    return df

In [10]:
#Data is our data frame with all the companies with first column 1 or 0 if Elon Musk talks about it.
# after,Name of the company and the other four covariates 
def compare(company_quote, date, companies, tickers, shares, hist): 

    company = compareCompanies(companies, tickers, date, shares, hist)

    company ['Elon']= 0
    company ['compare'] = company_quote
    company.at[4,'Elon'] = 1
    company.at[4,'compare'] = 'None'
    return company
    
#Call this function for all our functions and a set of company that will represent the controled one.

In [11]:
def create_data(companies_list):
    Company = companies_list
    company_quotes = Company [-1]
    quotes = elon_org_df[elon_org_df['ORG'] == company_quotes]
    quotes = quotes.sort_values('date', axis = 0)
    quotes = quotes['date']
    q = []
    for i in range(len(quotes)):
        q.append(quotes.iloc[i][0:10])
    quotes = pd.Series(q).unique()
    first_date = quotes[0]
    ticker =[ticker_of_company(Company[0]),
             ticker_of_company(Company[1]),
             ticker_of_company(Company[2]),
             ticker_of_company(Company[3]),
             ticker_of_company(Company[4])] 
    shares=[yf.Ticker(ticker[0]).info['sharesOutstanding'],
            yf.Ticker(ticker[1]).info['sharesOutstanding'],
            yf.Ticker(ticker[2]).info['sharesOutstanding'],
            yf.Ticker(ticker[3]).info['sharesOutstanding'],
            yf.Ticker(ticker[4]).info['sharesOutstanding']]
    hist =[yf.Ticker(ticker[0]).history(start=first_date)[['Close', 'Volume']],
           yf.Ticker(ticker[1]).history(start=first_date)[['Close', 'Volume']],
           yf.Ticker(ticker[2]).history(start=first_date)[['Close', 'Volume']],
           yf.Ticker(ticker[3]).history(start=first_date)[['Close', 'Volume']],
           yf.Ticker(ticker[4]).history(start=first_date)[['Close', 'Volume']]]
    data = pd.DataFrame(columns = ['Name','Date','Close','Volume','MarketCap','Popularity','Elon','compare'])
    for i in range (len(quotes)):
        date = quotes [i] 
        company = compare(company_quotes, date, Company, ticker, shares, hist)
        frame = [data, company]
        data = pd.concat(frame)
    data['Money Volume'] = data['Volume']*data['Close']
    return data

In [7]:
#Apple: 
data_Apple = create_data(['Microsoft','IBM','Samsung','Dell','Apple'] )
print (data_Apple.shape)

(220, 9)


In [16]:
#PayPal : 
data_PayPal = create_data(['Western Union','EuroNet','MoneyGram','Payoneer','PayPal'])

402007008
15.266165733337402
52856000
56.5
91688400
8.489999771118164
339008000
9.680000305175781
1174930048
36.709999084472656
402007008
15.05325984954834
52856000
64.30999755859375
91688400
8.890000343322754
339008000
9.680000305175781
1174930048
37.5099983215332
402007008
15.325908660888672
52856000
77.86000061035156
91688400
9.359999656677246
339008000
9.680000305175781
1174930048
32.630001068115234
402007008
14.601903915405273
52856000
73.36000061035156
91688400
7.860000133514404
339008000
9.680000305175781
1174930048
36.31999969482422
402007008
16.640453338623047
52856000
72.5199966430664
91688400
7.539999961853027
339008000
9.680000305175781
1174930048
37.41999816894531
402007008
17.248245239257812
52856000
77.25
91688400
6.849999904632568
339008000
9.680000305175781
1174930048
38.099998474121094
402007008
17.584997177124023
52856000
77.8499984741211
91688400
7.329999923706055
339008000
9.680000305175781
1174930048
38.11000061035156
402007008
16.137189865112305
52856000
90.73000

In [17]:
#Twitter : 
data_Twitter = create_data(['Google','Facebook','Snapchat','Pinterest','Twitter'])

317737984
829.0800170898438
2366279936
137.1699981689453
1355490048
27.09000015258789
562700032
24.399999618530273
798126976
15.75
317737984
953.4000244140625
2366279936
150.67999267578125
1355490048
18.100000381469727
562700032
24.399999618530273
798126976
16.969999313354492
317737984
930.8300170898438
2366279936
169.86000061035156
1355490048
13.100000381469727
562700032
24.399999618530273
798126976
16.209999084472656
317737984
977.0
2366279936
172.5
1355490048
14.710000038146973
562700032
24.399999618530273
798126976
17.670000076293945
317737984
1154.050048828125
2366279936
204.74000549316406
1355490048
13.649999618530273
562700032
24.399999618530273
798126976
44.13999938964844
317737984
1183.8599853515625
2366279936
207.22999572753906
1355490048
13.069999694824219
562700032
24.399999618530273
798126976
44.2599983215332
317737984
1183.8599853515625
2366279936
207.22999572753906
1355490048
13.069999694824219
562700032
24.399999618530273
798126976
44.2599983215332
317737984
1198.800048

In [19]:
#Ford : 
data_Ford = create_data(['Toyota','BMW','General Motors','Stellantis','Ford'])

1397980032
113.36852264404297
601995008
76.61175537109375
1450669952
28.69990348815918
3131279872
9.129837036132812
3920790016
11.514986991882324
1397980032
93.33200073242188
601995008
62.63626480102539
1450669952
26.892854690551758
3131279872
6.815302848815918
3920790016
10.740830421447754
1397980032
95.26899719238281
601995008
59.648582458496094
1450669952
27.218280792236328
3131279872
5.836825847625732
3920790016
11.04280948638916
1397980032
100.26195526123047
601995008
67.17774200439453
1450669952
32.22370147705078
3131279872
9.65713882446289
3920790016
10.314847946166992
1397980032
98.19851684570312
601995008
66.11117553710938
1450669952
30.22690773010254
3131279872
9.197680473327637
3920790016
9.527705192565918
1397980032
98.52479553222656
601995008
65.95316314697266
1450669952
30.358104705810547
3131279872
9.28276538848877
3920790016
9.396514892578125
1397980032
97.4878158569336
601995008
67.55696105957031
1450669952
30.926607131958008
3131279872
9.299782752990723
3920790016
9.5

In [20]:
frame = [data_Apple,data_Twitter, data_Ford, data_PayPal]
data = pd.concat(frame)
data.to_csv('Data/FinalFeatures.csv.bz2' , index=False)

In [ ]:
#Tesla 
#data_Tesla = create_data(['Rivian','Lucid','VolksWagen','General Motors','Tesla'])f

In [21]:
data = pd.read_csv('Data/FinalFeatures.csv.bz2')
data

,Name,Date,Close,Volume,MarketCap,Popularity,Elon,compare,Money Volume
0,Microsoft,2015-02-05,37.501652,34616600.0,2.824463e+11,31,0,Apple,1.298180e+09
1,IBM,2015-02-05,108.493607,3406299.0,9.694153e+10,4,0,Apple,3.695617e+08
2,Samsung,2015-02-05,266.104340,8005050.0,1.588584e+12,100,0,Apple,2.130179e+09
3,Dell,2015-02-05,12.067676,271519.0,3.349842e+09,15,0,Apple,3.276603e+06
4,Apple,2015-02-05,27.105112,174826400.0,4.523193e+11,40,1,None,4.738689e+09
...,...,...,...,...,...,...,...,...,...
765,Western Union,2020-04-15,18.063158,4038100.0,7.261516e+09,14,0,PayPal,7.294084e+07
766,EuroNet,2020-04-15,81.879997,559400.0,4.327849e+09,0,0,PayPal,4.580367e+07
767,MoneyGram,2020-04-15,1.380000,425300.0,1.265300e+08,4,0,PayPal,5.869140e+05
768,Payoneer,2020-04-15,9.680000,1151400.0,3.281598e+09,1,0,PayPal,1.114555e+07


In [22]:
#Propensity score : 
def add_propensityscore (data):#so that if we want to change only the first line don't have to do it
    #With a column of one and zero if elon musk talked about you 
    #company = company_quote.extend(controled)
    #data = compareCompanies(company,date)
    #Normalise the features 
    data['Close'] = (data['Close'] - data['Close'].mean())/data['Close'].std()
    data['Volume']= (data['Volume']-data['Volume'].mean())/data['Volume'].std()
    data['Money Volume']= (data['Money Volume']-data['Money Volume'].mean())/data['Money Volume'].std()
    data['MarketCap']= (data['MarketCap']-data['MarketCap'].mean())/data['MarketCap'].std()
    data['Popularity']= (data['Popularity']-data['Popularity'].mean())/data['Popularity'].std()
    #Create the model
    mod = smf.logit(formula='Elon ~ Close* Volume + MarketCap + Popularity', data=data)
    res = mod.fit()
    data['Propensity_score'] = res.predict()

In [23]:
add_propensityscore(data)

Optimization terminated successfully.
         Current function value: 0.252382
         Iterations 15


/Users/selimajaoua/opt/anaconda3/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1810: RuntimeWarning: overflow encountered in exp


In [26]:
data[data['Name']== 'Microsoft']

,Name,Date,Close,Volume,MarketCap,Popularity,Elon,compare,Money Volume,Propensity_score
0,Microsoft,2015-02-05,-0.147925,0.216289,-0.126825,-0.212319,0,Apple,-0.079856,0.253187
5,Microsoft,2015-02-06,-0.147934,0.147844,-0.126837,-0.287312,0,Apple,-0.080912,0.214985
10,Microsoft,2015-02-09,-0.147891,0.111662,-0.126782,-0.287312,0,Apple,-0.081409,0.207395
15,Microsoft,2015-02-10,-0.147930,0.293414,-0.126832,-0.262314,0,Apple,-0.078688,0.250899
20,Microsoft,2015-02-12,-0.147662,0.335773,-0.126494,-0.262314,0,Apple,-0.077586,0.250842
25,Microsoft,2015-02-21,-0.147555,0.171912,-0.126359,-0.462297,0,Apple,-0.080025,0.159117
30,Microsoft,2015-02-24,-0.147584,0.113547,-0.126396,-0.287312,0,Apple,-0.080995,0.199475
35,Microsoft,2015-03-12,-0.148057,0.711106,-0.126992,-0.262314,0,Apple,-0.072632,0.347835
40,Microsoft,2015-05-06,-0.147092,0.181492,-0.125776,-0.187321,0,Apple,-0.079227,0.229708
45,Microsoft,2015-05-07,-0.146902,0.232118,-0.125536,-0.212319,0,Apple,-0.078086,0.225120


In [27]:
def get_similarity(propensity_score1, propensity_score2):
    '''Calculate similarity for instances with given propensity scores'''
    return 1-np.abs(propensity_score1-propensity_score2)

In [28]:
def matchingfunction(companies):
    treatment_df = data[data['Elon'] == 1]
    control_df   = data[data['Elon'] == 0]
    # Create an empty undirected graph
    G = nx.Graph()
    # Loop through all the pairs of instances
    for control_id, control_row in control_df.iterrows():
        for treatment_id, treatment_row in treatment_df.iterrows():
            if (control_row['compare']== treatment_row['Name']) and (control_row['Date']== treatment_row['Date']) :
                # Calculate the similarity 
                similarity = get_similarity(control_row['Propensity_score'],
                                    treatment_row['Propensity_score'])
                # Add an edge between the two instances weighted by the similarity between them
                G.add_weighted_edges_from([(control_id, treatment_id, similarity)])
    # Generate and return the maximum weight matching on the generated graph
    matching = nx.max_weight_matching(G)
    return matching

In [29]:
matching = matchingfunction(data)
matched = [i[0] for i in list(matching)]+ [i[1]for i in list(matching)]
balanced = data.iloc[matched]

## After matching : 


In [ ]:
data[data['Name']== 'Apple']

In [ ]:
treated = balanced.loc[balanced['Elon']==1]

In [ ]:
controled = balanced.loc[balanced['Elon']==0]

In [ ]:
treated

In [ ]:
controled 

In [ ]:
matching


In [ ]:
matched

In [ ]:
data_Apple[data_Apple['Date']=='2015-10-09']

In [ ]:
before =time.time()
quotes_Apple = elon_org_df[elon_org_df['ORG'] == 'Apple']
first_date = quotes_Apple.iloc[0]['date'][0:10]
Company_Apple =['Microsoft','IBM','Samsung','Dell','Apple'] 
ticker_Apple =[ticker_of_company('Microsoft'),
               ticker_of_company('IBM'),
               ticker_of_company('Samsung'),
               ticker_of_company('Dell'),
              ticker_of_company('Apple')] 
shares_Apple =[yf.Ticker(ticker_Apple[0]).info['sharesOutstanding'],
              yf.Ticker(ticker_Apple[1]).info['sharesOutstanding'],
              yf.Ticker(ticker_Apple[2]).info['sharesOutstanding'],
              yf.Ticker(ticker_Apple[3]).info['sharesOutstanding'],
               yf.Ticker(ticker_Apple[4]).info['sharesOutstanding']]
hist_Apple =[yf.Ticker(ticker_Apple[0]).history(start=first_date)[['Close', 'Volume']],
              yf.Ticker(ticker_Apple[1]).history(start=first_date)[['Close', 'Volume']],
              yf.Ticker(ticker_Apple[2]).history(start=first_date)[['Close', 'Volume']],
              yf.Ticker(ticker_Apple[3]).history(start=first_date)[['Close', 'Volume']],
               yf.Ticker(ticker_Apple[4]).history(start=first_date)[['Close', 'Volume']]]
after =time.time()
print('fetch time:',str(after-before))

In [ ]:
hist_Apple[0].iloc[0]

In [ ]:
hist_Apple[0].iloc[0].name

In [ ]:
hist_Apple[0].iloc[0].name.date() > datetime.date.fromisoformat('2015-12-04')

In [ ]:
for i in range(len(hist_Apple[0])):
    if hist_Apple[0].iloc[i].name.date() > datetime.date.fromisoformat('2015-12-12'):
        print(hist_Apple[0].iloc[i])
        break